In [1]:
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append(os.path.abspath('../src'))


In [14]:
from load_data import load_data_from_postgres, load_data_using_sqlalchemy
from utils import missing_values_table, convert_bytes_to_megabytes, outliers_table, fix_outlier



load data from postgres Database

In [ ]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
df = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")



load data from postgres Database using sqlalchemy

In [3]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL using SQLAlchemy
df = load_data_using_sqlalchemy(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

Successfully loaded the data


EDA on the extracted dataset

In [4]:
df.shape

(150001, 55)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [ ]:
df.columns

In [5]:
# List of relevant columns for user overview analysis
columns = [
    'IMSI', 'MSISDN/Number', 'IMEI', 'Handset Manufacturer', 'Handset Type',
    'Dur. (ms)', 'Start', 'End', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)',
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]

# Create the df DataFrame with the selected columns
df = df[columns].copy()

In [6]:
df.shape

(150001, 30)

In [7]:
# Desplay the number of missing Values in the data set per each columns
missing_values_table(df)


Your selected dataframe has 30 columns.
There are 16 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Avg RTT DL (ms),27829,18.6,float64
Avg RTT UL (ms),27812,18.5,float64
MSISDN/Number,1066,0.7,float64
IMEI,572,0.4,float64
Handset Manufacturer,572,0.4,object
Handset Type,572,0.4,object
IMSI,570,0.4,float64
Dur. (ms),1,0.0,float64
Start,1,0.0,object
End,1,0.0,object


In [11]:
# Desplay the number of outlayers Values in the dataset 
outliers_table(df)


Your selected dataframe has 30 columns.
Outlier information for numerical columns is displayed below.


,Column,Z-Score Outliers,IQR Outliers
0,IMSI,3,13250
1,MSISDN/Number,2,24130
2,IMEI,0,0
3,Dur. (ms),2214,7184
4,Activity Duration DL (ms),3558,26659
5,Activity Duration UL (ms),2958,26022
6,Total DL (Bytes),0,0
7,Total UL (Bytes),55,243
8,Social Media DL (Bytes),0,0
9,Social Media UL (Bytes),0,0


cleaning the data by using different techniques

In [8]:
df.dropna(subset=['MSISDN/Number'], inplace=True)
missing_values_table(df)

Your selected dataframe has 30 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Avg RTT DL (ms),27644,18.6,float64
Avg RTT UL (ms),27625,18.5,float64


Filling the missing Values with mean 

In [10]:
# Calculate mean values
mean_dl = df['Avg RTT DL (ms)'].mean()
mean_ul = df['Avg RTT UL (ms)'].mean()

# Fill missing values with mean
df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(mean_dl)
df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(mean_ul)


fixing outliers

In [11]:
# List of relevant columns to apply quartiles(0.95) to fix outliers
quartiles = [
    'Dur. (ms)',  'Activity Duration DL (ms)', 'Activity Duration UL (ms)',
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]


In [28]:
df = df.to_frame()  # Convert Series to DataFrame


In [ ]:
# Apply the fix_outlier function to each specified column

for column in quartiles:
    if column in df.columns:
        df = fix_outlier(df, column)

Formatting the data

In [31]:
# Byte to Megabyte conversion

byte_columns = [
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'YouTube DL (Bytes)', 'YouTube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)'
]

In [ ]:
# converting Byte to Megabyte conversion
for column in byte_columns:
    if column in df.columns:
        df[column] = df[column].apply(convert_bytes_to_megabytes)


# renaming the column
df.rename(columns=lambda x: x.replace('Bytes', 'Megabytes') if 'Bytes' in x else x, inplace=True)

In [36]:
#  milliseconds to seconds conversion
millisecond_columns = [
    'Dur. (ms)',
    'Activity Duration DL (ms)',
    'Activity Duration UL (ms)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)'
]

In [ ]:
# converting milliseconds to seconds
for column in millisecond_columns:
    if column in df.columns:
        df[column] = df[column].apply(convert_ms_to_seconds)

# renaming the column
df.rename(columns=lambda x: x.replace('(ms)', '(s)') if '(ms)' in x else x, inplace=True)

Top 10 handsets used by the customers

In [ ]:
df['Handset Type'].value_counts().head(10)


Top 3 handset manufacturers

In [ ]:
df['Handset Manufacturer'].value_counts().head(3)

Top 5 handsets per top 3 handset manufacturer

In [ ]:
# Identify the top 3 handset manufacturers
top_3_manufacturers = df['Handset Manufacturer'].value_counts().head(3).index

# Finding the top 5 handsets for each of the top 3 manufacturers
top_5_handsets_per_manufacturer = {}

for manufacturer in top_3_manufacturers:
    # Filtering the DataFrame for the current manufacturer
    manufacturer_data = [df['Handset Manufacturer'] == manufacturer]
    
    # Identifying the top 5 handsets for the current manufacturer
    top_5_handsets = manufacturer_data['Handset Type'].value_counts().head(5)
    
    # Storing the results in a dictionary
    top_5_handsets_per_manufacturer[manufacturer] = top_5_handsets

# Displaying the results
for manufacturer, handsets in top_5_handsets_per_manufacturer.items():
    print(f"\nTop 5 handsets for {manufacturer}:")
    print(handsets)

Task-1.1: Aggregate user the information 

In [ ]:
application_columns = [
    'Social Media DL (Megabytes)', 'Social Media UL (Megabytes)',
    'Youtube DL (Megabytes)', 'Youtube UL (Megabytes)',
    'Netflix DL (Megabytes)', 'Netflix UL (Megabytes)',
    'Google DL (Megabytes)', 'Google UL (Megabytes)',
    'Email DL (Megabytes)', 'Email UL (Megabytes)',
    'Gaming DL (Megabytes)', 'Gaming UL (Megabytes)',
    'Other DL (Megabytes)', 'Other UL (Megabytes)'
]

In [ ]:
user_aggregated_data = df.groupby(user_id_column).agg({
    'Dur. (s)': 'sum',  # Total session duration per user
    'IMSI': 'count',  # Number of xDR sessions per user
    'Total DL (Megabytes)': 'sum',  # Total download data per user
    'Total UL (Megabytes)': 'sum',  # Total upload data per user
    **{col: 'sum' for col in application_columns}  # Total data volume for each application
}).rename(columns={'IMSI': 'Number of xDR Sessions'})

In [ ]:
for app in ['Social Media', 'Youtube', 'Netflix', 'Google', 'Email', 'Gaming', 'Other']:
    user_aggregated_data[f'{app} Total Data (Megabytes)'] = user_aggregated_data[f'{app} DL (Megabytes)'] + user_aggregated_data[f'{app} UL (Megabytes)']


# Step 3: Display the aggregated data
user_aggregated_data.head()

Visualizing the data

In [ ]:
# 1. Distribution of Session Durations
plt.figure(figsize=(12, 6))
sns.histplot(user_aggregated_data['Dur. (s)'], bins=30, kde=True)
plt.title('Distribution of Session Durations')
plt.xlabel('Session Duration (s)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Distribution of Total Data Download/Upload
plt.figure(figsize=(12, 6))

# Plot Total Download Data
plt.subplot(1, 2, 1)
sns.histplot(user_aggregated_data['Total DL (Megabytes)'], bins=30, kde=True, color='blue')
plt.title('Distribution of Total Download Data')
plt.xlabel('Total Download Data (MB)')
plt.ylabel('Frequency')

In [ ]:
# Total Upload Data
plt.subplot(1, 2, 2)
sns.histplot(user_aggregated_data['Total UL (Megabytes)'], bins=30, kde=True, color='orange')
plt.title('Distribution of Total Upload Data')
plt.xlabel('Total Upload Data (MB)')
plt.ylabel('Frequency')

In [ ]:
# Total Data Volume by Application
heatmap_data = user_aggregated_data.filter(like='Total Data (Megabytes)').mean()

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data.values.reshape(1, -1), annot=True, cmap='coolwarm', xticklabels=heatmap_data.index, yticklabels=['Average'])
plt.title('Heatmap of Average Total Data Volume by Application')
plt.xlabel('Application')
plt.ylabel('Average')
plt.show()